## Running the PKRNN-2CM model
This notebook provides a step-by-step guide on running the PKRNN-2CM model. 
1. Firstly, run the model without simulation to obtain predicted concentrations. 
2. Secondly, run the model with a chosen simulation option. This step involves creating a simulated dataset, using the predicted values as the ground truth, and then running the inference PKRNN-2CM model. If you also intend to run the inference PKRNN-1CM model, please refer to the "master" branch and use the simulated dataset for training the PKRNN-1CM model.

In [ ]:
#import necessary packages
import torch, random
from train import run_model, set_seed

#set device
use_cuda = torch.cuda.is_available()
if use_cuda: torch.cuda.set_device(3)                  #change the number to your device
device = torch.device("cuda:3" if use_cuda else "cpu") #change the number to your device
#set seed
set_seed(100)

#set input and output files
inFile = 'dataset.h5'         # the dataset file
outFile = 'PKRNN_2CM_best.h5' # the output file containing the results of the predictions

#set the parameter dictionary
paramdict = {'real_vanco_to_feedback': True,
 'change_regularize': 'square',
 'eta1_var': 0.12,
 'eta2_var': 0.149, 
 'eta3_var': 0.416, 
 'scale_change_eta': 100}

### Run the PKRNN-2CM model without simulation

In [ ]:
RMSE_train, RMSE_valid, RMSE_test, best_epoch = run_model(inFile, device = device, paramdict = paramdict, dataFile = outFile)

### Run the inference PKRNN-2CM model with the selected simulation option

In [ ]:
simFile = 'results/PKRNN_2CM.pth' # the file we need to use for the simulation
#simulation options
loc = 'peak'
sim_type = 'add_all'
real_data = 'remove'

In [ ]:
#run the model with the selected simulation option
RMSE_train, RMSE_valid, RMSE_test, best_epoch = run_model(inFile, device = device, paramdict = paramdict, dataFile = outFile, 
                                                          simulation = True, loc = loc, sim_type = sim_type, real_data = real_data)

## Model evaluation 
The following cells illustrate how to evaluate the predicted concentrations from the inference models. 
1. The evaluation uses Root Mean Square Error (RMSE) as the metric to compare the simulated ground truth time-concentration curve with the predicted curve from either the PKRNN-1CM or PKRNN-2CM inference model. 
2. You have the flexibility to choose the evaluation focus on peak level, trough level, or both, this choice can be different with the simulation location choice.
3. Additionally, you may opt to evaluate the first half of the curve, the later half, or the whole curve based on your preferences.

In [ ]:
from evaluation import evaluation
#set files to read data and model
File = f'batches_{loc}_{sim_type}_{real_data}_step2.h5'
underlying_path = f'results/PKRNN_2CM_sim_{loc}_{sim_type}_{real_data}_step1.pth'

### Evaluate the inference PKRNN-1CM model

In [ ]:
inference_path = f'results/PKRNN_1CM_sim_{loc}_{sim_type}_{real_data}.pth'

In [ ]:
# Evaluate on peaks, only elvaulate the early half of the curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_1CM', eval_loc = 'peak', eval_type = 'early')
# Evaluate on troughs, evaluate the whole curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_1CM', eval_loc = 'trough', eval_type = 'whole')
# Evaluate on peaks and troughs, only evaluate the later half of the curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_1CM', eval_loc = 'both', eval_type = 'later')

### Evaluate the inference PKRNN-2CM model

In [ ]:
inference_path2 = f'results/PKRNN_2CM_sim_{loc}_{sim_type}_{real_data}.pth'

In [ ]:
# Evaluate on peaks, only elvaulate the early half of the curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_2CM', eval_loc = 'peak', eval_type = 'early')
# Evaluate on troughs, evaluate the whole curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_2CM', eval_loc = 'trough', eval_type = 'whole')
# Evaluate on peaks and troughs, only evaluate the later half of the curve
evaluation(File, underlying_path, inference_path, device, model_name = 'PKRNN_2CM', eval_loc = 'both', eval_type = 'later')